In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [3]:
#config_file = "configs/vgg_ssd300_voc0712_tdt4265_server.yaml"
#config_file = "configs/mnist_tdt4265_server_experimental.yaml"
config_file = "configs/train_rdd2020_random-mirror.yaml"
# config_file = "configs/train_rdd2020_server_experimental.yaml"
cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-04-20 10:50:30,804 SSD INFO: Loaded configuration file configs/train_rdd2020_random-mirror.yaml
2021-04-20 10:50:30,808 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'res_net'
        PRETRAINED: True
        OUT_CHANNELS: [128, 256, 512, 512, 256, 128]
        INPUT_CHANNELS: 3
    PRIORS:
        MIN_SIZES: [[15, 15], [45, 45], [111, 111], [162, 162], [213, 213], [264, 264]]
        MAX_SIZES: [[45, 45], [111, 111], [162, 162], [213, 213], [264, 264], [315, 315]]
        FEATURE_MAPS: [[38, 38], [19, 19], [10, 10], [5, 5], [3, 3], [1, 1]]
        STRIDES: [[8, 8], [16, 16], [30, 30], [60, 60], [100, 100], [300, 300]]
INPUT:
    IMAGE_SIZE: [300, 300]
DATASETS:
    TRAIN: ("rdd2020_train",)
    TEST: ("rdd2020_val", )
SOLVER:
    MAX_ITER: 120000
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 1e-3
OUTPUT_DIR: 'outputs/rdd2020_random_mirror'
DATASET_DIR: "datasets"
MODEL_SAVE_STEP: 2000
EVAL_STEP: 10000 
2021-04-20 10:50:30,809 SSD INFO: Running with config:
DATASETS

In [ ]:
!pwd 
model = start_train(cfg)

/notebooks/TDT4265-Project/assignment4/SSD
Detector initialized. Total Number of params:  21.62M
Backbone number of parameters: 20.84M
SSD Head number of parameters: 788.2K
SSDDetector(
  (backbone): ResNetBackbone(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

  0%|          | 0/366 [00:00<?, ?it/s]/notebooks/TDT4265-Project/assignment4/SSD/ssd/modeling/box_head/inference.py:43: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:760.)
  indices = torch.nonzero(scores > self.cfg.TEST.CONFIDENCE_THRESHOLD).squeeze(1)
100%|██████████| 366/366 [04:25<00:00,  1.28it/s]


2021-04-20 11:10:20,159 SSD.inference INFO: mAP: 0.1577
D00             : 0.1122
D10             : 0.0024
D20             : 0.3885
D40             : 0.1277

2021-04-20 11:10:21,244 SSD.trainer INFO: iter: 020010, lr: 0.00010, total_loss: 4.623 (4.837), reg_loss: 1.797 (1.886), cls_loss: 2.825 (2.951), time: 26.957 (0.147), eta: 4:05:47, mem: 2002M
2021-04-20 11:10:22,350 SSD.trainer INFO: iter: 020020, lr: 0.00010, total_loss: 4.407 (4.837), reg_loss: 1.661 (1.886), cls_loss: 2.746 (2.951), time: 0.111 (0.147), eta: 4:05:41, mem: 2002M
2021-04-20 11:10:23,445 SSD.trainer INFO: iter: 020030, lr: 0.00010, total_loss: 4.700 (4.836), reg_loss: 1.843 (1.886), cls_loss: 2.858 (2.951), time: 0.110 (0.147), eta: 4:05:35, mem: 2002M
2021-04-20 11:10:24,548 SSD.trainer INFO: iter: 020040, lr: 0.00010, total_loss: 4.499 (4.836), reg_loss: 1.756 (1.885), cls_loss: 2.743 (2.951), time: 0.110 (0.147), eta: 4:05:29, mem: 2002M
2021-04-20 11:10:25,668 SSD.trainer INFO: iter: 020050, lr: 0.00010, total

In [ ]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

2021-04-19 16:49:01,085 SSD INFO: Start evaluating...
2021-04-19 16:49:01,085 SSD INFO: Start evaluating...
2021-04-19 16:49:01,085 SSD INFO: Start evaluating...
2021-04-19 16:49:01,085 SSD INFO: Start evaluating...


NameError: name 'model' is not defined